In [1]:
import sys
import time
from os import listdir
from os.path import isfile, join
import string
import numpy as np
from sklearn.linear_model import LogisticRegression as LR
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from collections import OrderedDict
import pandas as pd
from nltk.corpus import stopwords

In [2]:
# imports needed and logging
import gzip
import gensim 
import logging
from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def get_file_names(path):
    return [path + "/" + f for f in listdir(path) if isfile(join(path, f))]

In [4]:
def file_to_tokens(file_name):#, min_word_size, stemmer=None):
    with open(file_name) as f:
        file_string = f.read().rstrip()

        return gensim.utils.simple_preprocess(file_string)
        #return file_string

In [5]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

## Train Word Embeddings


In [6]:
def build_data(pos_file_path, neg_file_path ):
    pos_file_names = get_file_names(pos_file_path)
    neg_file_names = get_file_names(neg_file_path)
    # m is the number of training examples
    m_pos = len(pos_file_names)
    m_neg = len(neg_file_names)
    m = m_pos + m_neg
    print(m)
    pos_labels = np.ones(m_pos)
    neg_labels = np.zeros(m_neg)
    y = np.concatenate((pos_labels, neg_labels), axis=0)
    pos_data=pd.DataFrame(index=list(range(m_pos)),columns=['Review'])
    neg_data=pd.DataFrame(index=list(range(m_neg)),columns=['Review'])
    for i in range(m_pos):
        #pos_data.loc[i,'Review']=text_process(file_to_tokens(pos_file_names[i]))
        pos_data.loc[i,'Review']=file_to_tokens(pos_file_names[i])
    pos_data['Label'] = pos_labels
    for i in range(m_neg):
        #neg_data.loc[i,'Review']=text_process(file_to_tokens(neg_file_names[i]))
        neg_data.loc[i,'Review']=file_to_tokens(neg_file_names[i])
    neg_data['Label'] = neg_labels
    data=pd.concat([pos_data,neg_data],axis=0)
    data=data.sample(frac=1).reset_index(drop=True)
    return data



In [7]:
data_small = build_data('review_polarity_data/pos', 'review_polarity_data/neg' )
data_big_part1 = build_data ('aclImdb/train/pos','aclImdb/train/neg')
data_big_part2 = build_data ('aclImdb/test/pos','aclImdb/test/neg')
data_big = pd.concat([data_big_part1,data_big_part2],axis=0)
data_big = data_big.sample(frac=1).reset_index(drop=True)
data_huge = pd.concat([data_small,data_big],axis=0)
data_huge = data_huge.sample(frac=1).reset_index(drop=True)


2000
25000
25000


In [8]:
len(data_huge)

52000

In [9]:
# build vocabulary and train model
model = gensim.models.Word2Vec(
    data_huge['Review'],
    size=100,
    window=10,
    min_count=2,
    workers=5)
model.train(data_huge['Review'], total_examples=len(data_huge['Review']), epochs=100)

2019-02-09 08:19:07,933 : INFO : collecting all words and their counts
2019-02-09 08:19:07,935 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-09 08:19:08,328 : INFO : PROGRESS: at sentence #10000, processed 2406964 words, keeping 53602 word types
2019-02-09 08:19:08,739 : INFO : PROGRESS: at sentence #20000, processed 4805138 words, keeping 71228 word types
2019-02-09 08:19:09,151 : INFO : PROGRESS: at sentence #30000, processed 7186538 words, keeping 83867 word types
2019-02-09 08:19:09,553 : INFO : PROGRESS: at sentence #40000, processed 9572711 words, keeping 94161 word types
2019-02-09 08:19:09,944 : INFO : PROGRESS: at sentence #50000, processed 11946187 words, keeping 102923 word types
2019-02-09 08:19:10,029 : INFO : collected 104557 word types from a corpus of 12431273 raw words and 52000 sentences
2019-02-09 08:19:10,030 : INFO : Loading a fresh vocabulary
2019-02-09 08:19:10,349 : INFO : effective_min_count=2 retains 65356 unique words (62%

2019-02-09 08:19:53,258 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 08:19:53,261 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 08:19:53,263 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 08:19:53,264 : INFO : EPOCH - 4 : training on 12431273 raw words (9482213 effective words) took 11.2s, 843744 effective words/s
2019-02-09 08:19:54,274 : INFO : EPOCH 5 - PROGRESS: at 8.02% examples, 783593 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:19:55,286 : INFO : EPOCH 5 - PROGRESS: at 17.65% examples, 839106 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:19:56,287 : INFO : EPOCH 5 - PROGRESS: at 27.14% examples, 858474 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:19:57,298 : INFO : EPOCH 5 - PROGRESS: at 36.57% examples, 866739 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:19:58,318 : INFO : EPOCH 5 - PROGRESS: at 44.74% examples, 843907 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:19:59,328 

2019-02-09 08:20:41,727 : INFO : EPOCH 4 - PROGRESS: at 37.17% examples, 881726 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:20:42,735 : INFO : EPOCH 4 - PROGRESS: at 46.16% examples, 872749 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:20:43,747 : INFO : EPOCH 4 - PROGRESS: at 55.35% examples, 869509 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:20:44,751 : INFO : EPOCH 4 - PROGRESS: at 64.60% examples, 870787 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:20:45,761 : INFO : EPOCH 4 - PROGRESS: at 73.99% examples, 872161 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:20:46,776 : INFO : EPOCH 4 - PROGRESS: at 83.15% examples, 870269 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:20:47,781 : INFO : EPOCH 4 - PROGRESS: at 91.56% examples, 861486 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:20:48,725 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-09 08:20:48,736 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 08:20:48,739 : I

2019-02-09 08:21:33,556 : INFO : EPOCH 9 - PROGRESS: at 8.82% examples, 860213 words/s, in_qsize 10, out_qsize 1
2019-02-09 08:21:34,565 : INFO : EPOCH 9 - PROGRESS: at 18.13% examples, 863653 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:21:35,575 : INFO : EPOCH 9 - PROGRESS: at 27.50% examples, 869707 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:21:36,583 : INFO : EPOCH 9 - PROGRESS: at 37.17% examples, 879450 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:21:37,586 : INFO : EPOCH 9 - PROGRESS: at 46.76% examples, 882396 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:21:38,587 : INFO : EPOCH 9 - PROGRESS: at 56.08% examples, 882666 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:21:39,600 : INFO : EPOCH 9 - PROGRESS: at 65.55% examples, 883028 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:21:40,603 : INFO : EPOCH 9 - PROGRESS: at 74.93% examples, 883764 words/s, in_qsize 7, out_qsize 2
2019-02-09 08:21:41,609 : INFO : EPOCH 9 - PROGRESS: at 84.60% examples, 886469 words/s, in_qsiz

2019-02-09 08:22:26,293 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 08:22:26,299 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 08:22:26,300 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 08:22:26,302 : INFO : EPOCH - 13 : training on 12431273 raw words (9481887 effective words) took 10.7s, 883488 effective words/s
2019-02-09 08:22:27,308 : INFO : EPOCH 14 - PROGRESS: at 8.97% examples, 872396 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:22:28,311 : INFO : EPOCH 14 - PROGRESS: at 18.77% examples, 895462 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:22:29,321 : INFO : EPOCH 14 - PROGRESS: at 28.43% examples, 900426 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:22:30,327 : INFO : EPOCH 14 - PROGRESS: at 36.55% examples, 867475 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:22:31,334 : INFO : EPOCH 14 - PROGRESS: at 46.25% examples, 873588 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:22:3

...

2019-02-09 08:36:06,219 : INFO : EPOCH 87 - PROGRESS: at 21.76% examples, 683606 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:07,250 : INFO : EPOCH 87 - PROGRESS: at 27.74% examples, 652100 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:08,256 : INFO : EPOCH 87 - PROGRESS: at 34.78% examples, 654674 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:09,271 : INFO : EPOCH 87 - PROGRESS: at 39.94% examples, 626150 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:10,285 : INFO : EPOCH 87 - PROGRESS: at 46.25% examples, 619011 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:11,296 : INFO : EPOCH 87 - PROGRESS: at 52.19% examples, 611228 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:12,315 : INFO : EPOCH 87 - PROGRESS: at 58.38% examples, 607911 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:13,322 : INFO : EPOCH 87 - PROGRESS: at 65.77% examples, 616059 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:36:14,333 : INFO : EPOCH 87 - PROGRESS: at 73.49% examples, 625986 words/s

2019-02-09 08:37:00,153 : INFO : EPOCH 91 - PROGRESS: at 14.73% examples, 707101 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:01,160 : INFO : EPOCH 91 - PROGRESS: at 21.51% examples, 679599 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:02,172 : INFO : EPOCH 91 - PROGRESS: at 28.58% examples, 678090 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:03,210 : INFO : EPOCH 91 - PROGRESS: at 34.12% examples, 642962 words/s, in_qsize 10, out_qsize 1
2019-02-09 08:37:04,230 : INFO : EPOCH 91 - PROGRESS: at 40.52% examples, 634385 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:05,231 : INFO : EPOCH 91 - PROGRESS: at 46.17% examples, 618777 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:37:06,235 : INFO : EPOCH 91 - PROGRESS: at 53.01% examples, 621675 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:07,239 : INFO : EPOCH 91 - PROGRESS: at 60.93% examples, 636312 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:08,273 : INFO : EPOCH 91 - PROGRESS: at 69.03% examples, 645985 words/

2019-02-09 08:37:52,754 : INFO : EPOCH 95 - PROGRESS: at 18.86% examples, 890507 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:37:53,792 : INFO : EPOCH 95 - PROGRESS: at 27.74% examples, 864503 words/s, in_qsize 9, out_qsize 2
2019-02-09 08:37:54,794 : INFO : EPOCH 95 - PROGRESS: at 34.34% examples, 806712 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:37:55,795 : INFO : EPOCH 95 - PROGRESS: at 40.12% examples, 754576 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:56,798 : INFO : EPOCH 95 - PROGRESS: at 46.00% examples, 719914 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:57,828 : INFO : EPOCH 95 - PROGRESS: at 51.51% examples, 688530 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:58,834 : INFO : EPOCH 95 - PROGRESS: at 57.28% examples, 671291 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:37:59,860 : INFO : EPOCH 95 - PROGRESS: at 63.32% examples, 658273 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:00,863 : INFO : EPOCH 95 - PROGRESS: at 70.31% examples, 658162 words/s

2019-02-09 08:38:46,657 : INFO : EPOCH 99 - PROGRESS: at 39.94% examples, 754976 words/s, in_qsize 10, out_qsize 0
2019-02-09 08:38:47,658 : INFO : EPOCH 99 - PROGRESS: at 48.47% examples, 761518 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:48,662 : INFO : EPOCH 99 - PROGRESS: at 56.96% examples, 767848 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:49,676 : INFO : EPOCH 99 - PROGRESS: at 64.84% examples, 763170 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:50,678 : INFO : EPOCH 99 - PROGRESS: at 72.84% examples, 762221 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:51,689 : INFO : EPOCH 99 - PROGRESS: at 81.21% examples, 764475 words/s, in_qsize 8, out_qsize 1
2019-02-09 08:38:52,698 : INFO : EPOCH 99 - PROGRESS: at 89.58% examples, 766493 words/s, in_qsize 9, out_qsize 0
2019-02-09 08:38:53,706 : INFO : EPOCH 99 - PROGRESS: at 97.59% examples, 765136 words/s, in_qsize 10, out_qsize 0
2019-02-09 08:38:53,975 : INFO : worker thread finished; awaiting finish of 4 more thr

(948276988, 1243127300)

In [10]:
model.save("word2vec4_huge_data_100,10,2,5,100.model")

2019-02-09 08:39:06,009 : INFO : saving Word2Vec object under word2vec4_huge_data_100,10,2,5,50.model, separately None
2019-02-09 08:39:06,013 : INFO : not storing attribute vectors_norm
2019-02-09 08:39:06,015 : INFO : not storing attribute cum_table
2019-02-09 08:39:07,040 : INFO : saved word2vec4_huge_data_100,10,2,5,50.model


In [11]:
model = Word2Vec.load("word2vec4_huge_data_100,10,2,5,100.model")

2019-02-09 08:39:07,264 : INFO : loading Word2Vec object from word2vec4_huge_data_100,10,2,5,50.model
2019-02-09 08:39:07,997 : INFO : loading wv recursively from word2vec4_huge_data_100,10,2,5,50.model.wv.* with mmap=None
2019-02-09 08:39:07,998 : INFO : setting ignored attribute vectors_norm to None
2019-02-09 08:39:07,999 : INFO : loading vocabulary recursively from word2vec4_huge_data_100,10,2,5,50.model.vocabulary.* with mmap=None
2019-02-09 08:39:08,000 : INFO : loading trainables recursively from word2vec4_huge_data_100,10,2,5,50.model.trainables.* with mmap=None
2019-02-09 08:39:08,000 : INFO : setting ignored attribute cum_table to None
2019-02-09 08:39:08,001 : INFO : loaded word2vec4_huge_data_100,10,2,5,50.model


## NLP_Task : now let's use our pretrained word embedding to accomplish our task

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier,Lasso
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,f1_score, auc, roc_auc_score
from sklearn.model_selection import ShuffleSplit

/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [13]:
word_vectors = model.wv

In [14]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


## Pre-trained glove model

In [15]:
with open("glove.6B/glove.6B.200d.txt", "rb") as lines:
    w2v_100 = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}
    
encoding="utf-8"
import struct 

glove_small = {}
all_words = set(w for words in data_huge['Review'] for w in words)
with open("glove.6B/glove.6B.200d.txt", "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if (word in all_words):
            nums=np.array(parts[1:], dtype=np.float32)
            glove_small[word] = nums


## Classifiers

In [16]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [18]:
data_huge_train, data_huge_test, label_huge_train, label_huge_test = \
train_test_split(data_huge['Review'], data_huge['Label'], test_size=0.2, random_state = 0)
print(len(data_huge_train), len(data_huge_test), len(data_huge_train) + len(data_huge_test))

41600 10400 52000


In [19]:

pipeline1 = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),  # strings to token integer counts
    ('classifier', RidgeClassifier(random_state=0))
    #('classifier', xgb.XGBClassifier(random_state=0))
])



In [20]:
pipeline2 = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(glove_small)),  # strings to token integer counts
    ('classifier', RidgeClassifier(random_state=0))
])

In [22]:
pipeline1.fit(data_huge_train,label_huge_train)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7fb03ee94978>), ('classifier', RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=0, solver='auto',
        tol=0.001))])

In [23]:
predictions1 = pipeline1.predict(data_huge_test)

In [24]:
print(roc_auc_score(predictions1,label_huge_test))

0.8795685513811701


In [25]:
print(classification_report(predictions1,label_huge_test))

             precision    recall  f1-score   support

        0.0       0.87      0.89      0.88      5163
        1.0       0.89      0.87      0.88      5237

avg / total       0.88      0.88      0.88     10400



In [28]:
predictions2 = pipeline2.predict(data_huge_test)

In [29]:
print(classification_report(predictions2,label_huge_test))

             precision    recall  f1-score   support

        0.0       0.83      0.84      0.84      5196
        1.0       0.84      0.83      0.84      5204

avg / total       0.84      0.84      0.84     10400



In [35]:
print(roc_auc_score(predictions2,label_huge_test))


0.8359649916952614


In [ ]:
predictionsTrain = pipeline2.predict(data_big_train)
print(classification_report(predictionsTrain,label_big_train))

In [ ]:
label_train

In [ ]:
predictions

In [ ]:
label_train